# Mistral Models on Amazon Bedrock - Comprehensive Test Suite

This notebook tests 7 key Mistral models available on Amazon Bedrock:
1. **Mistral Large 3** - Flagship multimodal model with reasoning
2. **Ministral 3B** - Efficient small model
3. **Ministral 8B** - Mid-size efficient model
4. **Ministral 14B** - Larger efficient model
5. **Voxtral Mini 3B** - Audio transcription model
6. **Voxtral Small 24B** - Advanced audio model
7. **Magistral Small 1.2** - Advanced reasoning model with vision (24B)

In [ ]:
%pip install --upgrade --quiet boto3 soundfile

In [ ]:
import boto3
import json
import base64
from io import BytesIO

# Initialize Bedrock client
bedrock_runtime = boto3.client('bedrock-runtime', region_name='us-west-2')

## Helper Functions

In [ ]:
def invoke_mistral(model_id, messages, max_tokens=2048, temperature=0.7, tools=None):
    """Invoke Mistral models with messages format"""
    body = {
        "messages": messages,
        "max_tokens": max_tokens,
        "temperature": temperature,
        # "response_format": {"type": "json_object"}  # output in JSON format
    }
    if tools:
        body["tools"] = tools
    
    response = bedrock_runtime.invoke_model(
        modelId=model_id,
        body=json.dumps(body)
    )
    return json.loads(response['body'].read())

def print_response(response):
    """Pretty print model response"""
    if 'content' in response:
        for content in response['content']:
            if content['type'] == 'text':
                print(content['text'])
            elif content['type'] == 'tool_use':
                print(f"Tool: {content['name']}")
                print(f"Input: {json.dumps(content['input'], indent=2)}")
    elif 'choices' in response:
        print(response['choices'][0]['message']['content'])
    else:
        print(json.dumps(response, indent=2))

## 1. Mistral Large 3 - Flagship Model Tests

### 1.1 Basic Text Generation

In [ ]:
model_id = "mistral.mistral-large-3-675b-instruct"

messages = [{
    "role": "user",
    "content": "Explain quantum computing in 3 sentences."
}]

response = invoke_mistral(model_id, messages, max_tokens=500)
print("=== Mistral Large 3 - Basic Generation ===")
print_response(response)

### 1.2 Reasoning Use Case with Extended Thinking

In [ ]:
# Enable reasoning mode for step-by-step thinking
messages = [{
    "role": "user",
    "content": "A bat and ball cost $1.10 total. The bat costs $1 more than the ball. How much does the ball cost? Think step by step."
}]

# Use lower temperature for reasoning tasks
response = invoke_mistral(model_id, messages, max_tokens=1000, temperature=0.3)
print("\n=== Mistral Large 3 - Reasoning ===")
print_response(response)

### 1.3 Tool Use / Function Calling

**Note:** Tool use with invoke_model has compatibility issues. Use the **Converse API** notebook instead for reliable tool calling.

In [ ]:
# Define tools
tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current weather for a location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City name"
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"]
                }
            },
            "required": ["location"]
        }
    }
}]

messages = [{
    "role": "user",
    "content": "What's the weather in Paris and Tokyo?"
}]

response = invoke_mistral(model_id, messages, tools=tools)
print("\n=== Mistral Large 3 - Tool Use ===")
print_response(response)

### 1.4 Multi-turn Conversation

In [ ]:
messages = [
    {"role": "user", "content": "What are the three laws of robotics?"},
    {"role": "assistant", "content": "The Three Laws of Robotics by Isaac Asimov are: 1) A robot may not injure a human or allow harm through inaction, 2) A robot must obey human orders unless conflicting with First Law, 3) A robot must protect itself unless conflicting with First or Second Law."},
    {"role": "user", "content": "Give me a scenario where these laws conflict."}
]

response = invoke_mistral(model_id, messages)
print("\n=== Mistral Large 3 - Multi-turn ===")
print_response(response)

### 1.6 Vision with Reasoning

Combine vision with step-by-step reasoning.

In [ ]:
# # Load image from local repo
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


# Path to your image
image_path = "Battle.png"

# Getting the Base64 string
base64_image = encode_image(image_path)


messages = [{
    "role": "user",
    "content": [
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/png;base64,{base64_image}"
            }
                    },
        {
            "type": "text",
            "text": "What is in this image? Describe it in detail."
        }
    ]
}]

response = invoke_mistral(model_id, messages, max_tokens=1000)
print("\n=== Mistral Large 3 - Vision (Image) ===")
print_response(response)

## 2. Ministral 3B - Efficient Small Model

In [ ]:
model_id = "mistral.ministral-3-3b-instruct"

# Basic generation
messages = [{"role": "user", "content": "Write a Python function to calculate fibonacci numbers."}]
response = invoke_mistral(model_id, messages, max_tokens=500)
print("=== Ministral 3B - Code Generation ===")
print_response(response)

In [ ]:
# Tool use test
tools = [{
    "type": "function",
    "function": {
        "name": "calculate",
        "description": "Perform mathematical calculation",
        "parameters": {
            "type": "object",
            "properties": {
                "expression": {"type": "string", "description": "Math expression"}
            },
            "required": ["expression"]
        }
    }
}]

messages = [{"role": "user", "content": "What is 234 * 567?"}]
response = invoke_mistral(model_id, messages, tools=tools)
print("\n=== Ministral 3B - Tool Use ===")
print_response(response)

In [ ]:
response

In [ ]:
messages = [{
    "role": "user",
    "content": [
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/png;base64,{base64_image}"
            }
                    },
        {
            "type": "text",
            "text": "What is in this image? Describe it in detail."
        }
    ]
}]

response = invoke_mistral(model_id, messages, max_tokens=1000)
print("\n=== Ministral 3B - Vision (Image) ===")
print_response(response)


## 3. Ministral 8B - Mid-Size Model

In [ ]:
model_id = "mistral.ministral-3-8b-instruct"

# Text analysis task
messages = [{
    "role": "user",
    "content": "Analyze the sentiment and extract key entities from: 'Apple Inc. released iPhone 15 in September 2023, receiving positive reviews from tech enthusiasts.'"
}]

response = invoke_mistral(model_id, messages)
print("=== Ministral 8B - NLP Task ===")
print_response(response)


In [ ]:
# Tool use with multiple tools
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_database",
            "description": "Search customer database",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string"}
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "send_email",
            "description": "Send email to customer",
            "parameters": {
                "type": "object",
                "properties": {
                    "to": {"type": "string"},
                    "subject": {"type": "string"},
                    "body": {"type": "string"}
                },
                "required": ["to", "subject", "body"]
            }
        }
    }
]

messages = [{"role": "user", "content": "Find customer John Doe and send him a reminder email about his pending order."}]
response = invoke_mistral(model_id, messages, tools=tools)
print("\n=== Ministral 8B - Multi-Tool Use ===")
print_response(response)

In [ ]:
response

In [ ]:
print(model_id)
messages = [{
    "role": "user",
    "content": [
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/png;base64,{base64_image}"
            }
                    },
        {
            "type": "text",
            "text": "What is in this image? Describe it in detail."
        }
    ]
}]

response = invoke_mistral(model_id, messages, max_tokens=1000)
print("\n=== Ministral 8B - Vision (Image) ===")
print_response(response)

## 4. Ministral 14B - Larger Efficient Model

In [ ]:
model_id = "mistral.ministral-3-14b-instruct"

# Complex reasoning task
messages = [{
    "role": "user",
    "content": "Design a database schema for an e-commerce platform with users, products, orders, and reviews. Explain your design choices."
}]

response = invoke_mistral(model_id, messages, max_tokens=1500)
print("=== Ministral 14B - Complex Task ===")
print_response(response)


In [ ]:
# Tool use with complex parameters
tools = [{
    "type": "function",
    "function": {
        "name": "query_analytics",
        "description": "Query analytics database",
        "parameters": {
            "type": "object",
            "properties": {
                "metrics": {"type": "array", "items": {"type": "string"}},
                "dimensions": {"type": "array", "items": {"type": "string"}},
                "date_range": {
                    "type": "object",
                    "properties": {
                        "start": {"type": "string"},
                        "end": {"type": "string"}
                    }
                }
            },
            "required": ["metrics"]
        }
    }
}]

messages = [{"role": "user", "content": "Get me the revenue and user signups by country for last month."}]
response = invoke_mistral(model_id, messages, tools=tools)
print("\n=== Ministral 14B - Complex Tool Use ===")
print_response(response)

In [ ]:
response

In [ ]:
print(model_id)
messages = [{
    "role": "user",
    "content": [
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/png;base64,{base64_image}"
            }
                    },
        {
            "type": "text",
            "text": "What is in this image? Describe it in detail."
        }
    ]
}]

response = invoke_mistral(model_id, messages, max_tokens=1000)
print("\n=== Ministral 14B - Vision (Image) ===")
print_response(response)

## 5. Voxtral Models - Audio Transcription

✅ **Voxtral audio models WORK with proper audio format!**

**Required audio format:**
- Mono channel (1 channel)
- 16kHz sample rate
- MP3 format with low bitrate (32k)

Use the helper function below to convert any audio file to the correct format.

In [ ]:
def convert_audio_for_voxtral(audio_file_or_bytes):
    """Convert audio to Voxtral-compatible format: mono, 16kHz, MP3"""
    from pydub import AudioSegment
    import io
    
    # Load audio (handles MP3, WAV, OGG, etc.)
    if isinstance(audio_file_or_bytes, bytes):
        audio = AudioSegment.from_file(io.BytesIO(audio_file_or_bytes))
    else:
        # Detect format from file extension
        if audio_file_or_bytes.endswith('.mp3'):
            audio = AudioSegment.from_mp3(audio_file_or_bytes)
        elif audio_file_or_bytes.endswith('.ogg'):
            audio = AudioSegment.from_ogg(audio_file_or_bytes)
        elif audio_file_or_bytes.endswith('.wav'):
            audio = AudioSegment.from_wav(audio_file_or_bytes)
        else:
            audio = AudioSegment.from_file(audio_file_or_bytes)
    
    # Convert to mono (1 channel)
    audio_mono = audio.set_channels(1)
    
    # Convert to 16kHz sample rate
    audio_mono = audio_mono.set_frame_rate(16000)
    
    # Export to MP3 bytes with low bitrate
    mp3_io = io.BytesIO()
    audio_mono.export(mp3_io, format='mp3', bitrate='32k')
    
    return mp3_io.getvalue()


def invoke_voxtral(model_id, audio_data, prompt="", max_tokens=2048):
    """Invoke Voxtral models with audio input - WORKING FORMAT"""
    # Encode audio to base64
    audio_base64 = base64.b64encode(audio_data).decode('utf-8')
    
    # Build content with audio
    content = [{
        "type": "input_audio",
        "input_audio": {
            "data": audio_base64,
            "format": "mp3"
        }
    }]
    
    # Add text prompt if provided
    if prompt:
        content.append({
            "type": "text",
            "text": prompt
        })
    
    body = {
        "messages": [{
            "role": "user",
            "content": content
        }],
        "max_tokens": max_tokens
    }
    
    response = bedrock_runtime.invoke_model(
        modelId=model_id,
        body=json.dumps(body)
    )
    return json.loads(response['body'].read())

In [ ]:
!pip install pydub

In [ ]:
!sudo apt update
!sudo apt install ffmpeg -y

### 5.1 Voxtral Mini - Basic Audio Transcription

In [ ]:
# Example with obama.mp3
print("=== Voxtral Mini 3B - Audio Transcription ===")

# Convert audio to proper format
audio_data = convert_audio_for_voxtral('obama.mp3')
print(f"Audio converted: {len(audio_data)} bytes")

# Transcribe the audio
model_id = 'mistral.voxtral-mini-3b-2507'
response = invoke_voxtral(model_id, audio_data, prompt="Transcribe this audio exactly as spoken.")
print_response(response)

### 5.2 Voxtral Mini - Transcription with Custom Prompt

In [ ]:
print("=== Voxtral Mini 3B - Custom Prompt ===")

# Load and convert audio
audio_data = convert_audio_for_voxtral('jfk.wav')

# Transcribe with custom prompt
response = invoke_voxtral(
    model_id='mistral.voxtral-mini-3b-2507',
    audio_data=audio_data,
    prompt='Transcribe this audio and identify the main theme of the speech.'
)
print_response(response)

## 6. Voxtral Small 24B - Advanced Audio

Voxtral Small provides higher quality transcription for complex audio.

In [ ]:
print("=== Voxtral Small 24B - High-Quality Transcription ===")

# Convert audio to proper format
audio_data = convert_audio_for_voxtral('obama.mp3')

# Use larger model for better quality
model_id = 'mistral.voxtral-small-24b-2507'
response = invoke_voxtral(model_id, audio_data,  prompt="Transcribe this audio exactly as spoken.")
print_response(response)

### 6.2 Voxtral Small - Audio Analysis with Detailed Prompts

In [ ]:
print("=== Voxtral Small 24B - Audio Analysis ===")

audio_data = convert_audio_for_voxtral('obama.mp3')

response = invoke_voxtral(
    model_id='mistral.voxtral-small-24b-2507',
    audio_data=audio_data,
    prompt='Transcribe and analyze: 1) Speaker tone and emotion, 2) Key messages, 3) Main themes'
)
print_response(response)

## 7. Magistral Small 1.2 - Advanced Reasoning Model

Magistral Small is a 24B parameter model specialized for complex reasoning with vision capabilities.

### 7.1 Advanced Reasoning

Magistral uses special <reasoning> tokens to show reasoning process.

In [ ]:
def invoke_magistral_vision(model_id, image_data, prompt, max_tokens=2048):
    """Invoke Magistral with vision + reasoning - CORRECT FORMAT"""
    # Encode image to base64
    image_base64 = base64.b64encode(image_data).decode('utf-8')
    
    system_prompt = """First draft your thinking process (inner monologue) until you arrive at a response. 
        Format your response using Markdown, and use LaTeX for any mathematical equations. 
        Write both your thoughts and the response in the same language as the input.
        
        Your thinking process must follow the template below:[THINK]Your thoughts or/and draft, 
        like working through an exercise on scratch paper. Be as casual and as long as you want until you are confident to generate the response. 
        Use the same language as the input.[/THINK]Here, provide a self-contained response.
        """
    # CORRECT format: image_url must be an object with "url" property
    body = {
        "messages": [
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": system_prompt
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/png;base64,{image_base64}"
                        }
                    },
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ],
        "max_tokens": max_tokens,
        "temperature": 0.7
    }
    
    response = bedrock_runtime.invoke_model(
        modelId=model_id,
        body=json.dumps(body)
    )
    return json.loads(response['body'].read())

In [ ]:
# Load image from local repo
with open('Battle.png', 'rb') as f:
    image_data = f.read()

print("\n=== Magistral Small - Vision Reasoning ===")
response = invoke_magistral_vision(
    model_id='mistral.magistral-small-2509',
    image_data=image_data,
    prompt='What action do you think I should take in this situation? List all the possible actions and explain why you think they are good or bad.'
)
print_response(response)

In [ ]:
response

## Summary

This notebook demonstrates:

**Text Models:**
- **Mistral Large 3**: Complex reasoning, multi-turn conversations, **vision (images)**, advanced tool use
- **Ministral 3B/8B/14B**: Efficient models for coding, NLP, tool use with increasing capability

**Specialized Models:**
- **Voxtral Mini/Small**: Audio transcription ✅ **WORKS** (mono, 16kHz)
- **Magistral Small 1.2**: Advanced reasoning with <THINK> tokens, **multimodal vision** capabilities

**Key Capabilities Tested:**
- Basic text generation
- Reasoning and step-by-step thinking
- Tool use / function calling
- Multi-turn conversations
- **Vision - Image understanding and analysis** (Mistral Large 3, Magistral)
- **Vision with reasoning** (Magistral with [THINK] tokens)
- **Document/chart analysis** (both models)
- **Audio transcription** ✅ (Voxtral Mini/Small with proper format)
- Advanced reasoning with visible thought process ([THINK] tokens)
- Multimodal reasoning (vision + text)
- Optimized reasoning configurations (top_p=0.95, temperature=0.7)


**Audio Requirements:**
For Voxtral models, use `convert_audio_for_voxtral()` function:
- Converts to mono (1 channel)
- Converts to 16kHz sample rate
- Exports as MP3 with 32k bitrate